# Perceptrón multicapa
## Para el curso de Inteligencia Artificial
### Alan García Zermeño
12 de marzo de 2023


---



In [ ]:
import numpy as np 

#Vector One-Hot
def onehot(v,mdi=None):
  if mdi is None:
    mdi = v.max()+1
  b = np.zeros((v.size,mdi))
  b[np.arange(v.size), v] = 1
  return b

#Entropía cruzada
def CrEntropia(pr,trues):
  epsilon = 1e-12
  pr = np.clip(pr, epsilon, 1-epsilon)
  ce = -np.sum(trues*np.log(pr+epsilon))/pr.shape[0]
  return ce

#Función de activación
def activacion(x):
  return 1.0/(1.0+np.exp(-x))

In [ ]:
#Front-propagation
def F_propagation(x,W1,W2):    
  z = activacion(np.dot(x, W1))
  pog = activacion(np.dot(z,W2))
  return pog,z

#Back-propagation
def B_propagation(x,y,learning_rate,output_s,pog,z,W1,W2):
  #ultima capa
  aux = onehot(y,mdi = output_s)
  delta_last = aux - pog      
  grad_W2 = np.dot(z.T,delta_last)
        
  #Capa oculta
  grad_tmp = np.multiply(z,(1-z))
  b = np.dot(delta_last,W2.T)

  Hdelta = np.multiply(b,grad_tmp)
  grad_W1 = np.dot(x.T,Hdelta)/x.shape[0]
  grad_W2 = grad_W2/x.shape[0]

  W1 += learning_rate*grad_W1       
  W2 += learning_rate*grad_W2
  return W1,W2

#Loss
def perdida(x,y,W1,W2):
  tmp,z = F_propagation(x,W1,W2)
  return CrEntropia(tmp,onehot(y))

#-------------------------MLP-------------------------#
def MLperceptron(x,y,epochs,batch_size,learning_rate):
  hidden_s = 128
  input_s = 4
  output_s = 3
  W1 = np.random.randn(input_s,hidden_s)
  W2 = np.random.randn(hidden_s,output_s)  

  for e in range(epochs):
    if e%10 == 0:
      print(e, "----->","Perdida:",perdida(x,y,W1,W2))

    idxs = np.random.permutation(len(x)) #shuffle

    for i in range(0,len(x),batch_size):
      idx = idxs[i:i+batch_size]
      xx = x[idx]
      yy = y[idx]
      
      pog,z = F_propagation(xx,W1,W2)
      W1,W2 = B_propagation(xx,yy,learning_rate,output_s,pog,z,W1,W2)

  return W1,W2

## Tomamos los datos del dataset de Iris, con tres clases distintas y usamos el perceptrón multicapa creado para entrenar un 80% de los datos y testear con un 20%.

In [ ]:
from sklearn.datasets import load_iris

iris = load_iris()
X = iris.data
Y = iris.target

idx = np.random.permutation(Y.size)
aux =  int(0.2*Y.size)

X_train = X[idx[aux:],:]
X_test = X[idx[:aux],:]
Y_train = Y[idx[aux:]]
Y_test = Y[idx[:aux]]

In [ ]:
W1,W2 = MLperceptron(X_train, Y_train, epochs=300, batch_size = 10, learning_rate = 1e-2)

0 -----> Perdida: 0.0004844928521391868
10 -----> Perdida: 0.4566370338576269
20 -----> Perdida: 0.3910276102927728
30 -----> Perdida: 0.33304190705233855
40 -----> Perdida: 0.3055907702942063
50 -----> Perdida: 0.2890645215028086
60 -----> Perdida: 0.280883508513037
70 -----> Perdida: 0.27689423937827357
80 -----> Perdida: 0.2477647016639983
90 -----> Perdida: 0.23696380783064958
100 -----> Perdida: 0.2383793897513628
110 -----> Perdida: 0.229260888023191
120 -----> Perdida: 0.21791699659367156
130 -----> Perdida: 0.21185392196507571
140 -----> Perdida: 0.21369456950115975
150 -----> Perdida: 0.19233575053031954
160 -----> Perdida: 0.20197316196487017
170 -----> Perdida: 0.18552742242761944
180 -----> Perdida: 0.17520335087669128
190 -----> Perdida: 0.1770249573269367
200 -----> Perdida: 0.1687566404711528
210 -----> Perdida: 0.16899733137621203
220 -----> Perdida: 0.17008672127383845
230 -----> Perdida: 0.1612462433738257
240 -----> Perdida: 0.15711746646311384
250 -----> Perdida: 0.

In [ ]:
pred,z = F_propagation(X_test,W1,W2)
pred = np.array([np.argmax(x) for x in pred])
print("Accuracy: ",sum(pred==Y_test)/len(pred))

Accuracy:  1.0


# Usando 300 épocas, una tasa de aprendizaje de 1e-2 y un batch size de 10, podemos alzanzar un 100% de precisión en los datos de prueba.